In [1]:
import math
import copy

In [5]:
class GaussianSolver:

    def __init__(self):
        self.operations_count = 0
    def reset_counter(self):
        self.operations_count = 0
    def add_operations(self, count=1):
        self.operations_count += count

    "метод гаусса"
    def solve(self, A, b, pivot_tipe = "none"):
        self.reset_counter()
        n = len(A)
        A = copy.deepcopy(A)
        b = copy.deepcopy(b)

        print(f"\nКоэффициенты СЛАУ {n}x{n}:")
        for i in range(n):
            equation = f"{A[i][0]:.1f}*x1"
            for j in range(1, n):
                equation += f" + {A[i][j]:.1f}*x{j+1}"
            equation += f" = {b[i]:.1f}"
            print(equation)

        for k in range(n-1):
            if pivot_tipe == "column":
                max_row = k
                for i in range(k+1, n):
                    if abs(A[i][k]) > abs(A[max_row][k]):
                        max_row = i
                    self.add_operations(2)
                if max_row != k:
                    A[k], A[max_row] = A[max_row], A[k]
                    b[k], b[max_row] = b[max_row], b[k]
                    self.add_operations(2)
            elif pivot_tipe == 'row':
                max_col = k
                for j in range(k+1, n):
                    if abs(A[k][j]) > abs(A[k][max_col]):
                        max_col = j
                    self.add_operations(2)
                if max_col != k:
                    for i in range(n):
                        A[i][k], A[i][max_col] = A[i][max_col], A[i][k]
                    self.add_operations(n)
            elif pivot_tipe == 'full':
                max_row, max_col = k, k
                max_val = abs(A[k][k])
                for i in range(k, n):
                    for j in range(k, n):
                        if abs(A[i][j]) > max_val:
                            max_val = abs(A[i][j])
                            max_row, max_col = i, j
                        self.add_operations(2)
                if max_row != k:
                    A[k], A[max_row] = A[max_row], A[k]
                    b[k], b[max_row] = b[max_row], b[k]
                    self.add_operations(2)
                if max_col != k:
                    for i in range(n):
                        A[i][k], A[i][max_col] = A[i][max_col], A[i][k]
                    self.add_operations(n)

            for i in range(k+1, n):
                multiplier = A[i][k] / A[k][k]
                self.add_operations(1)
                for j in range(k+1, n):
                    A[i][j] -= multiplier * A[k][j]
                    self.add_operations(2)
                b[i] -= multiplier * b[k]
                self.add_operations(2)

        x = [0,0] * n
        for i in range(n-1, -1, -1):
            sum_ax = 0.0
            for j in range(i+1, n):
                sum_ax += A[i][j] * x[j]
                self.add_operations(2)
            x[i] = (b[i] - sum_ax) / A[i][i]
            self.add_operations(2)
        return x

    "определитель"
    def determinant(self, A):
        self.reset_counter()
        n = len(A)
        A_copy = copy.deepcopy(A)
        det = 1.0
        for k in range(n-1):
            for i in range(k+1, n):
                multiplier = A_copy[i][k] / A_copy[k][k]
                self.add_operations(1)
                for j in range(k, n):
                    A_copy[i][j] -= multiplier * A_copy[k][j]
                    self.add_operations(2)
            det *= A_copy[k][k]
            self.add_operations(1)
        det *= A_copy[n-1][n-1]
        self.add_operations(1)
        return det
    def theoretical_operations(self, n):
        return (2/3) * n**3 + (3/2) * n**2 - (7/6) * n

In [6]:
"матрица гильберта"
def create_hilbert_matrix(n):
    A = [[0.0] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            A[i][j] = 1.0 / (i + j + 1)
    return A

In [7]:
"тестим"
def main():
    print("ЗАДАНИЕ 1: МЕТОД ГАУССА")
    print("=" * 50)

    solver = GaussianSolver()

    A = [
        [2.0, 1.0, -1.0],
        [-3.0, -1.0, 2.0],
        [-2.0, 1.0, 2.0]
    ]
    b = [8.0, -11.0, -3.0]

    print("Исходная система:")
    print("Матрица A =", A)
    print("Вектор b =", b)

    print("\n1. Предварительная оценка системы:")
    det = solver.determinant(A)
    print(f"   Определитель матрицы: {det:.4f}")
    if abs(det) < 1e-10:
        print("   ВЫВОД: Система может быть вырожденной")
    else:
        print("   ВЫВОД: Система имеет единственное решение")

    print("\n2. Решение с выбором главного элемента:")
    strategies = ['none', 'column', 'row', 'full']

    for strategy in strategies:
        print(f"\n   --- Стратегия: {strategy} ---")
        x = solver.solve(A, b, strategy)
        print(f"   Решение: x = {[f'{val:.4f}' for val in x]}")
        print(f"   Количество операций: {solver.operations_count}")

    print("\n3. Сравнение количества операций:")
    n = len(A)
    theoretical = solver.theoretical_operations(n)
    practical = solver.operations_count
    print(f"   Теоретически: {theoretical:.0f}")
    print(f"   Практически: {practical}")
    print(f"   Отношение: {practical/theoretical:.2f}")

    print(f"\n4. Определитель матрицы коэффициентов: {det:.4f}")

    print("\n" + "=" * 50)
    print("ЗАДАНИЕ 2: МАТРИЦА ГИЛЬБЕРТА")
    print("=" * 50)

    sizes = [3, 5, 7]

    for n in sizes:
        print(f"\n--- n = {n} ---")

        A_hilbert = create_hilbert_matrix(n)
        print(f"Матрица Гильберта {n}x{n}:")
        for i in range(n):
            print(f"  Строка {i+1}: {[f'{A_hilbert[i][j]:.4f}' for j in range(n)]}")

        exact_solution = [1.0] * n
        print(f"Точное решение: x = {exact_solution}")

        b_hilbert = [0.0] * n
        for i in range(n):
            for j in range(n):
                b_hilbert[i] += A_hilbert[i][j] * exact_solution[j]
        print(f"Правая часть: b = {[f'{val:.4f}' for val in b_hilbert]}")

        print(f"\nРешение системы A*x = b:")
        x_calculated = solver.solve(A_hilbert, b_hilbert, 'column')
        print(f"Найденное решение: x = {[f'{val:.4f}' for val in x_calculated]}")

        error = 0.0
        for i in range(n):
            error += (x_calculated[i] - exact_solution[i]) ** 2
        error = math.sqrt(error)
        print(f"Погрешность: {error:.2e}")

        det = solver.determinant(A_hilbert)
        print(f"Определитель матрицы Гильберта: {det:.2e}")

if __name__ == "__main__":
    main()

ЗАДАНИЕ 1: МЕТОД ГАУССА
Исходная система:
Матрица A = [[2.0, 1.0, -1.0], [-3.0, -1.0, 2.0], [-2.0, 1.0, 2.0]]
Вектор b = [8.0, -11.0, -3.0]

1. Предварительная оценка системы:
   Определитель матрицы: -1.0000
   ВЫВОД: Система имеет единственное решение

2. Решение с выбором главного элемента:

   --- Стратегия: none ---

Коэффициенты СЛАУ 3x3:
2.0*x1 + 1.0*x2 + -1.0*x3 = 8.0
-3.0*x1 + -1.0*x2 + 2.0*x3 = -11.0
-2.0*x1 + 1.0*x2 + 2.0*x3 = -3.0
   Решение: x = ['2.0000', '3.0000', '-1.0000', '0.0000', '0.0000', '0.0000']
   Количество операций: 31

   --- Стратегия: column ---

Коэффициенты СЛАУ 3x3:
2.0*x1 + 1.0*x2 + -1.0*x3 = 8.0
-3.0*x1 + -1.0*x2 + 2.0*x3 = -11.0
-2.0*x1 + 1.0*x2 + 2.0*x3 = -3.0
   Решение: x = ['2.0000', '3.0000', '-1.0000', '0.0000', '0.0000', '0.0000']
   Количество операций: 41

   --- Стратегия: row ---

Коэффициенты СЛАУ 3x3:
2.0*x1 + 1.0*x2 + -1.0*x3 = 8.0
-3.0*x1 + -1.0*x2 + 2.0*x3 = -11.0
-2.0*x1 + 1.0*x2 + 2.0*x3 = -3.0
   Решение: x = ['2.0000', '3.0000', '